In [ ]:
import cv2
import numpy as np
import glob
import os.path

def align_images(im1, im2):
    
    # Convert images to grayscale
    im1_gray = cv2.cvtColor(im1,cv2.COLOR_BGR2GRAY)
    im2_gray = cv2.cvtColor(im2,cv2.COLOR_BGR2GRAY)
 
    # Find size of image1
    sz = im1.shape
 
    # Define the motion model
    warp_mode = cv2.MOTION_TRANSLATION
 
    # Define 2x3 or 3x3 matrices and initialize the matrix to identity
    if warp_mode == cv2.MOTION_HOMOGRAPHY :
        warp_matrix = np.eye(3, 3, dtype=np.float32)
    else :
        warp_matrix = np.eye(2, 3, dtype=np.float32)
 
    # Specify the number of iterations.
    number_of_iterations = 5000;
 
    # Specify the threshold of the increment
    # in the correlation coefficient between two iterations
    termination_eps = 1e-10;
 
    # Define termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, number_of_iterations,  termination_eps)
 
    # Run the ECC algorithm. The results are stored in warp_matrix.
    (cc, warp_matrix) = cv2.findTransformECC(im1_gray,im2_gray,warp_matrix, warp_mode, criteria)
 
    if warp_mode == cv2.MOTION_HOMOGRAPHY :
    # Use warpPerspective for Homography
        im2_aligned = cv2.warpPerspective(im2, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP)
    else :
    # Use warpAffine for Translation, Euclidean and Affine
        im2_aligned = cv2.warpAffine(im2, warp_matrix, (sz[1],sz[0]), flags=cv2.INTER_LINEAR + cv2.WARP_INVERSE_MAP);

    return im2_aligned
        
if __name__ == '__main__':   
    
    names = list(glob.iglob(r'data directory*.jpg'))
    
    print(imFilename)
    # Read reference image
    refFilename = "ref.jpg"
    print("Reading reference image : ", refFilename)
    imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)

    for imFilename in names[1:]:
        
        # Read image to be aligned
        print("Reading image to align : ", imFilename);
        im = cv2.imread(imFilename, cv2.IMREAD_COLOR)

        print("Aligning images ...")
        # Registered image will be resotred in im2_aligned.
        im2_aligned = align_images(imReference, im)

        # Write aligned image to disk.
        outFilenamerep = imFilename.replace('.jpg', '_aligned.jpg')
        outFilename = os.path.join('aligned directory', os.path.basename(outFilenamerep))
        print("Saving aligned image : ", outFilename);
        cv2.imwrite(outFilename, im2_aligned)
        
        cv2.waitKey(0)